In [1]:
%pip install keras-tuner
%pip install keras_self_attention

     |████████████████████████████████| 97 kB 2.9 MB/s 
  Created wheel for keras-self-attention: filename=keras_self_attention-0.50.0-py3-none-any.whl size=19414 sha256=91edd5fb637cb34958d63cae1784362aa6758723925cd62043094f29c51dd5e9
  Stored in directory: /root/.cache/pip/wheels/92/7a/a3/231bef5803298e7ec1815215bc0613239cb1e9c03c57b13c14
Successfully built keras-self-attention


In [2]:
import site
import os
import tensorflow as tf
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
import errno
import numpy as np
import itertools
import multiprocessing
import json
import datetime
from collections import defaultdict
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute, Reshape, Lambda, RepeatVector, Input, Multiply, SimpleRNN, GRU, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.summary import create_file_writer

from keras_self_attention import SeqSelfAttention, SeqWeightedAttention

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 40)

In [3]:
def coShuffled_vectors(X, Y):
    if tf.shape(X)[0] == tf.shape(Y)[0]:
        test_idxs = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
        shuffled_test_idxs = tf.random.shuffle(test_idxs)
        return (tf.gather(X, shuffled_test_idxs), tf.gather(Y, shuffled_test_idxs))
    else:
        raise ValueError(f"0-dimension has to be the same {tf.shape(X)[0]} != {tf.shape(Y)[0]}")


def getNpArrayFromH5(hf_Data):
    X_train = hf_Data['Train_Data']  # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Data['Label']  # Get train label
    Y_train = np.array(Y_train)
    return X_train, Y_train

# data extraction
def getData(is500=True, shuffle=False, ise2e=False, include_secondary=False, validation_split=None, isColab=False):
    if not include_secondary:
        hf_Train = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/{"e2e_Train_Data" if ise2e else "Fold_10_Train_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
        hf_Test = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/{"e2e_Test_Data" if ise2e else "Fold_10_Test_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
    else:
        hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Secondary_Data_1136.h5', 'r')
        hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Secondary_Data_1136.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    X_validation = Y_validation = None
    if validation_split is not None:
        # sklearn split shuffles anyway
        X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_split)

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getE2eData(is500=True, shuffle=False, include_secondary=False, isColab=False):
    if not include_secondary:
        hf_Train = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Data_{str(500) if is500 else str(1000)}.h5', 'r')
        hf_Test = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Data_{str(500) if is500 else str(1000)}.h5', 'r')
    else:
        hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Secondary_Data_1136.h5', 'r')
        hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Secondary_Data_1136.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_Secondary_Data_1136.h5', 'r') if include_secondary else h5.File(
        f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_Data_{str(500) if is500 else str(1000)}.h5', 'r')
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, 13)  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getE2eDataJustSecondary(shuffle=False,isColab=False):
    hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_just_Secondary_Data_1000.h5', 'r')
    hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_just_Secondary_Data_1000.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_just_Secondary_Data_1000.h5', 'r')
    
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, 13)  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


In [4]:
def plot_history(history):
    acc_keys = [k for k in history.history.keys() if k in ('accuracy', 'val_accuracy')]
    loss_keys = [k for k in history.history.keys() if not k in acc_keys]
    for k, v in history.history.items():
        if k in acc_keys:
            plt.figure(1)
            plt.plot(v)
        else:
            plt.figure(2)
            plt.plot(v)
    plt.figure(1)
    plt.title('Accuracy vs. epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(acc_keys, loc='lower right')
    plt.figure(2)
    plt.title('Loss vs. epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loss_keys, loc='upper right')
    plt.show()

def plot_history_df(history):
    acc_keys = [k for k in history.columns.values if k in ('accuracy', 'val_accuracy')]
    loss_keys = [k for k in history.columns.values if not k in acc_keys and not k in ['epoch']]
    for k, v in history.items():
        if k in acc_keys:
            plt.figure(1)
            plt.plot(v)
        if k in loss_keys:
            plt.figure(2)
            plt.plot(v)
    plt.figure(1)
    plt.title('Accuracy vs. epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(acc_keys, loc='lower right')
    plt.figure(2)
    plt.title('Loss vs. epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loss_keys, loc='upper right')
    plt.show()   

def get_layer_by_name(layers, name, return_first=True):
    matching_named_layers = [l for l in layers if l.name == name]
    if not matching_named_layers:
        return None
    return matching_named_layers[0] if return_first else matching_named_layers


def get_combined_features_from_models(
        to_combine,
        X_train_combined, Y_train_combined,
        X_validation_combined, Y_validation_combined,
        X_test_combined, Y_test_combined,
        reverse_one_hot=False,
        normalize_X_func=None):
    models = []
    models_dict = {}
    X_trains_out = []
    X_test_out = []
    X_validation_out = []
    XY_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

    if reverse_one_hot:
        Y_train_new = np.apply_along_axis(np.argmax, 1, Y_train_combined) + 1
        Y_test_new = np.apply_along_axis(np.argmax, 1, Y_test_combined) + 1
        Y_validation_new = np.apply_along_axis(np.argmax, 1, Y_validation_combined) + 1
    else:
        Y_train_new = Y_train_combined.copy()
        Y_test_new = Y_test_combined.copy()
        Y_validation_new = np.apply_along_axis(np.argmax, 1, Y_validation_combined) + 1

    for model_file_name, layer_name, kwargs in to_combine:
        model_here = None
        if isinstance(model_file_name, tf.keras.models.Model):
            model_here = model_file_name
            model_file_name = model_here.name
        else:
            if model_file_name in models_dict.keys():
                model_here = models_dict[model_file_name]
            else:
                model_here = tf.keras.models.load_model(model_file_name,
                                                        **kwargs) if kwargs is not None else tf.keras.models.load_model \
                    (model_file_name)

        features_model = Model(model_here.input,
                               get_layer_by_name(model_here.layers, layer_name).output)
        if normalize_X_func is None:
            X_trains_out.append(np.array(features_model.predict(X_train_combined), dtype='float64'))
            X_test_out.append(np.array(features_model.predict(X_test_combined), dtype='float64'))
            X_validation_out.append(np.array(features_model.predict(X_validation_combined), dtype='float64'))
        else:
            X_trains_out.append(np.array(normalize_X_func(features_model.predict(X_train_combined)), dtype='float64'))
            X_test_out.append(np.array(normalize_X_func(features_model.predict(X_test_combined)), dtype='float64'))
            X_validation_out.append(np.array(normalize_X_func(features_model.predict(X_validation_combined)), dtype='float64'))
            
        XY_dict[model_file_name][layer_name]['Train']['X'] = X_trains_out[-1]
        XY_dict[model_file_name][layer_name]['Train']['Y'] = Y_train_new
        
        XY_dict[model_file_name][layer_name]['Test']['X'] = X_test_out[-1]
        XY_dict[model_file_name][layer_name]['Test']['Y'] = Y_test_new
        
        XY_dict[model_file_name][layer_name]['Validation']['X'] = X_validation_out[-1]
        XY_dict[model_file_name][layer_name]['Validation']['Y'] = Y_validation_new
        
        
        models.append(((model_file_name, layer_name), (model_here, features_model)))
        models_dict[model_file_name] = model_here

    X_train_new = np.concatenate(tuple(X_trains_out), axis=1)
    X_test_new = np.concatenate(tuple(X_test_out), axis=1)
    X_validation_new = np.concatenate(tuple(X_validation_out), axis=1)


    data_train = (X_train_new, Y_train_new)
    data_test = (X_test_new, Y_test_new)
    data_validation = (X_validation_new, Y_validation_new)

    return models, data_train, data_validation, data_test, XY_dict



def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')


def source_model(model_func, model_name, input_shape):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
    return m


def compile_and_fit_model_with_tb(model_func,
                                  model_name,
                                  input_shape,
                                  X_train,
                                  Y_train,
                                  save_every_epoch=True,
                                  save_final=False,
                                  **kwargs):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
    tb_callback = TensorBoard(log_dir=f'{m.name}_logs', histogram_freq=kwargs.pop("histogram_freq", 1))
    if save_every_epoch:
        tb_callback.append(ModelCheckpoint(f'{m.name}' + '_model_{epoch:03d}_{val_accuracy:0.2f}'))
    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = m.fit(X_train, Y_train, callbacks=[tb_callback], verbose=2, **kwargs)
    if save_final:
        make_dir_if_not_exist(model_name)
        m.save(f"{m.name}_saved_model_after_fit")  # Save the model
    return (m, history)
    # m.save(f"{m.name}_Tenth_Fold_New_Model_500_8") #Save the model


def compile_model_and_fit_with_custom_loop(model_func,
                                           model_name,
                                           input_shape,
                                           X_train,
                                           Y_train,
                                           **kwargs):
    make_dir_if_not_exist(model_name)
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)

    train_writer = create_file_writer(f'{m.name}_logs/train/')
    test_writer = create_file_writer(f'{m.name}_logs/test/')
    train_step = test_step = 0

    acc_metric = tf.keras.metrics.CategoricalAccuracy()
    optimizer = tf.keras.optimizers.Adam()
    num_epochs = kwargs.get("epochs", 10)

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    BATCH_SIZE = kwargs.get("batch_size", 32)
    X_test, Y_test = kwargs.get("validation_data", (None, None))
    if X_test is None:
        raise ValueError("Missing X validation data")
    if Y_test is None:
        raise ValueError("Missing Y validation data")

    train_dataset_tf = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset_tf = train_dataset_tf.batch(BATCH_SIZE)
    train_dataset_tf = train_dataset_tf.prefetch(AUTOTUNE)

    test_dataset_tf = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
    test_dataset_tf = train_dataset_tf.batch(BATCH_SIZE)
    test_dataset_tf = train_dataset_tf.prefetch(AUTOTUNE)

    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    for epoch in range(num_epochs):
        # Iterate through training set
        for batch_idx, (x, y) in enumerate(train_dataset_tf):
            with tf.GradientTape() as tape:
                y_pred = m(x, training=True)
                loss = loss_fn(y, y_pred)

            gradients = tape.gradient(loss, m.trainable_weights)
            optimizer.apply_gradients(zip(gradients, m.trainable_weights))
            acc_metric.update_state(y, y_pred)

            with train_writer.as_default():
                tf.summary.scalar("Loss", loss, step=train_step)
                tf.summary.scalar(
                    "Accuracy", acc_metric.result(), step=train_step,
                )
                train_step += 1
        # Reset accuracy in between epochs (and for testing and test)
        acc_metric.reset_states()
        # Iterate through test set
        for batch_idx, (x, y) in enumerate(test_dataset_tf):
            y_pred = m(x, training=False)
            loss = loss_fn(y, y_pred)
            acc_metric.update_state(y, y_pred)
            with test_writer.as_default():
                tf.summary.scalar("Loss", loss, step=test_step)
                tf.summary.scalar(
                    "Accuracy", acc_metric.result(), step=test_step,
                )
                test_step += 1

        acc_metric.reset_states()  # Reset accuracy in between epochs (and for testing and test)

    return m


def reinitialize_weights(model):
    for ix, layer in enumerate(model.layers):
        if hasattr(model.layers[ix], 'kernel_initializer') and hasattr(model.layers[ix], 'bias_initializer'):
            weight_initializer = model.layers[ix].kernel_initializer
            bias_initializer = model.layers[ix].bias_initializer

            old_weights, old_biases = model.layers[ix].get_weights()

            model.layers[ix].set_weights([
                weight_initializer(shape=old_weights.shape),
                bias_initializer(shape=len(old_biases))])
    return model


def reverse_tensor(X):
    return tf.gather(X, tf.reverse(tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32), (0,)))


def run_mirrored_strategy(model_func, base_batch_size, nepochs, x_train, y_train, x_test, y_test, **kwargs):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = model_func()
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=tf.keras.metrics.SparseCategoricalAccuracy()
        )
    batch_size_mirr_strat = base_batch_size * strategy.num_replicas_in_sync
    history = model.fit(x_train, y_train, epochs=nepochs, batch_size=batch_size_mirr_strat,
                        validation_data=(x_test, y_test),
                        **kwargs)
    return model, history


def sparse_setdiff(a1, a2):
    a1a = a1.reshape(a1.shape[0], -1)
    a2a = a2.reshape(a2.shape[0], -1)
    spa2a = [np.where(x)[0].tolist() for x in a2a]
    spa1a = [np.where(x)[0].tolist() for x in a1a]
    idxs_to_keep = []
    for idx, sample in enumerate(spa1a):
        try:
            spa2a.index(sample)
        except ValueError:
            # not in list
            idxs_to_keep.append(idx)
    return a1[idxs_to_keep], idxs_to_keep


def unpacking_apply_along_axis(all_args):
    """
    Like numpy.apply_along_axis(), but with arguments in a tuple
    instead.

    This function is useful with multiprocessing.Pool().map(): (1)
    map() only handles functions that take a single argument, and (2)
    this function can generally be imported from a module, as required
    by map().
    """
    (func1d, axis, arr, args, kwargs) = all_args
    # return np.apply_along_axis(func1d, axis, arr, *args, **kwargs)


def parallel_apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """
    Like numpy.apply_along_axis(), but takes advantage of multiple
    cores.
    """
    # Effective axis where apply_along_axis() will be applied by each
    # worker (any non-zero axis number would work, so as to allow the use
    # of `np.array_split()`, which is only done on axis 0):
    effective_axis = 1 if axis == 0 else axis
    if effective_axis != axis:
        arr = arr.swapaxes(axis, effective_axis)

    # Chunks for the mapping (only a few chunks):
    chunks = [(func1d, effective_axis, sub_arr, args, kwargs)
              for sub_arr in np.array_split(arr, multiprocessing.cpu_count())]

    pool = multiprocessing.Pool()
    individual_results = pool.map(unpacking_apply_along_axis, chunks)
    # Freeing the workers:
    pool.close()
    pool.join()

    return np.concatenate(individual_results)


In [5]:
def model_A_CNN_256(model_name, inshape, num_classes = 13):


    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv1D(256 ,10 ,padding='same' ,input_shape=inshape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
    model.add(tf.keras.layers.MaxPooling1D(2))

    model.add(tf.keras.layers.GaussianNoise(1))
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Conv1D(256 ,10 ,padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
    model.add(tf.keras.layers.MaxPooling1D(4))

    model.add(tf.keras.layers.GaussianNoise(1))
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Conv1D(256 ,10 ,padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
    model.add(tf.keras.layers.MaxPooling1D(2))

    # check these 2
    model.add(tf.keras.layers.GaussianNoise(1))
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Conv1D(256 ,10 ,padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
    model.add(tf.keras.layers.MaxPooling1D(4))

    model.add(tf.keras.layers.GaussianNoise(1))
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Conv1D(256 ,10 ,padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
    model.add(tf.keras.layers.MaxPooling1D(4))

    model.add(tf.keras.layers.GaussianNoise(1))
    model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

    model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
    model._name = model_name

    return model


In [6]:
def model_E_RNN(model_name, input_shape = (1000, 8,),num_classes = 13):

    # RNN part
    inputs = Input(shape=input_shape)
    lstm_one = Bidirectional \
        (GRU(256, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(inputs)
    lstm_two = Bidirectional \
        (GRU(128, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(lstm_one)
    attention = SeqWeightedAttention()(lstm_two)
    attention = Flatten()(attention)
    rnnoutput = Dense(256 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(attention)
    rnnoutput = BatchNormalization()(rnnoutput)
    rnnoutput = GaussianNoise(1)(rnnoutput)
    rnnoutput = Dropout(0.4)(rnnoutput)

    # Dense Feed-forward
    dense_one = Dense(128, kernel_initializer='RandomNormal', bias_initializer='zeros')(rnnoutput)
    dense_one = LeakyReLU()(dense_one)
    dense_one = Dropout(0.5)(dense_one)
    dense_one = BatchNormalization()(dense_one)
    dense_two = Dense(64, kernel_initializer='RandomNormal', bias_initializer='zeros')(dense_one)
    dense_two = LeakyReLU()(dense_two)
    dense_two = Dropout(0.4)(dense_two)

    # Output
    output = Dense(num_classes, activation='softmax')(dense_two)
    model = Model([inputs], output, name = model_name)
    return model


In [7]:
def model_combination(model_name, input_shape,num_classes = 13):
    model = Sequential([
        tf.keras.Input(shape=input_shape),
        BatchNormalization(),
        Dense(256, kernel_initializer='RandomNormal', bias_initializer='zeros'),
        LeakyReLU(),
        Dropout(0.6),
        Dense(128, kernel_initializer='RandomNormal', bias_initializer='zeros', kernel_regularizer = tf.keras.regularizers.l1(1e-3)),
        LeakyReLU(),
        Dropout(0.6),
        Dense(32, kernel_initializer='RandomNormal', bias_initializer='zeros', kernel_regularizer = tf.keras.regularizers.l1(1e-2)),
        LeakyReLU(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ], name=model_name)
    return model

In [9]:
def merged_model_A_E(model_name, input_shape, num_classes = 13):
  base_input = Input(shape=input_shape, name='base_input')

  cnn_convblock_1 = MaxPooling1D(2)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(base_input))))
  cnn_noiseblock_1 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_1))

  cnn_convblock_2 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_1))))
  cnn_noiseblock_2 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_2))

  cnn_convblock_3 = MaxPooling1D(2)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_2))))
  cnn_noiseblock_3 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_3))

  cnn_convblock_4 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_3))))
  cnn_noiseblock_4 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_4))

  cnn_convblock_5 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_4))))
  cnn_noiseblock_5 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_5))

  cnn_flatten = Flatten()(cnn_noiseblock_5)

  cnn_denseblock_1  = LeakyReLU(alpha=0.5)(BatchNormalization()(Dense(128)(cnn_flatten)))
  cnn_denseblock_2  = LeakyReLU(alpha=0.5)(BatchNormalization()(Dense(64)(cnn_denseblock_1)))

  rnn_lstm_one = Bidirectional \
      (GRU(256, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(base_input)
  rnn_lstm_two = Bidirectional \
      (GRU(128, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(rnn_lstm_one)
  
  rnn_attention = Flatten()(SeqWeightedAttention()(rnn_lstm_two))

  rnn_denseblock_1 = BatchNormalization()(Dense(256 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_attention))
  rnn_noiseblock_1 = Dropout(0.4)(GaussianNoise(1)(rnn_denseblock_1))

  rnn_denseblock_2 = LeakyReLU()(Dense(128 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_noiseblock_1))
  rnn_noiseblock_2 = BatchNormalization()(Dropout(0.5)(rnn_denseblock_2))

  rnn_denseblock_3 = LeakyReLU()(Dense(64 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_noiseblock_2))
  rnn_noiseblock_3 = Dropout(0.4)(rnn_denseblock_3)

  merged_layer = tf.keras.layers.concatenate([cnn_denseblock_2,rnn_noiseblock_3])

  # Output
  output = Dense(num_classes, activation='softmax')(merged_layer)
  model = Model(base_input, output, name = model_name)

  return model




In [8]:
def sequential_model_A_E(model_name, input_shape, num_classes = 13):
  
  base_input = Input(shape=input_shape, name='base_input')

  cnn_convblock_1 = MaxPooling1D(2)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(base_input))))
  cnn_noiseblock_1 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_1))

  cnn_convblock_2 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_1))))
  cnn_noiseblock_2 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_2))

  cnn_convblock_3 = MaxPooling1D(2)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_2))))
  cnn_noiseblock_3 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_3))

  cnn_convblock_4 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_3))))
  cnn_noiseblock_4 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_4))

  cnn_convblock_5 = MaxPooling1D(4)(LeakyReLU(alpha=0.5)(BatchNormalization()(Conv1D(256 ,10 ,padding='same')(cnn_noiseblock_4))))
  cnn_noiseblock_5 = Dropout(rate=0.5)(GaussianNoise(1)(cnn_convblock_5))

  rnn_lstm_one = Bidirectional \
      (GRU(256, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(cnn_noiseblock_5)
  rnn_lstm_two = Bidirectional \
      (GRU(128, return_sequences=True, kernel_initializer='RandomNormal', dropout= 0.5, recurrent_dropout = 0.5, recurrent_initializer='RandomNormal', bias_initializer='zero'))(rnn_lstm_one)
  
  rnn_attention = Flatten()(SeqWeightedAttention()(rnn_lstm_two))

  rnn_denseblock_1 = BatchNormalization()(Dense(256 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_attention))
  rnn_noiseblock_1 = Dropout(0.4)(GaussianNoise(1)(rnn_denseblock_1))

  rnn_denseblock_2 = LeakyReLU()(Dense(128 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_noiseblock_1))
  rnn_noiseblock_2 = BatchNormalization()(Dropout(0.5)(rnn_denseblock_2))

  rnn_denseblock_3 = LeakyReLU()(Dense(64 ,kernel_initializer='RandomNormal', bias_initializer='zeros')(rnn_noiseblock_2))
  rnn_noiseblock_3 = Dropout(0.4)(rnn_denseblock_3)

  # merged_layer = tf.keras.layers.concatenate([cnn_denseblock_2,rnn_noiseblock_3])

  # Output
  output = Dense(num_classes, activation='softmax')(rnn_noiseblock_3)
  model = Model(base_input, output, name = model_name)

  return model


In [10]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

def compile_and_fit_model_basic(  model_func,
                                  model_name,
                                  input_shape,
                                  X_train,
                                  Y_train,
                                  save_max_epoch=True,
                                  save_final=False,
                                  patience_count = None,
                                  early_stopping_obs = 'accuracy',
                                  log_history = True,
                                  **kwargs):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)

    callbacks_used = []
    if save_max_epoch:
        callbacks_used.append(ModelCheckpoint(f'/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}' + '_model_{epoch:03d}_{accuracy:0.3f}',
                                              save_weights_only=False,
                                              monitor = early_stopping_obs, # 'accuracy', # 'val_accuracy'
                                              mode='max',
                                              save_best_only=True))
    if patience_count is not None:
        callbacks_used.append(tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=patience_count))

    if log_history:
        callbacks_used.append(tf.keras.callbacks.CSVLogger(f"/content/drive/MyDrive/ncRNA/data/history/history_log_{model_name}_{datetime.date.today().strftime('%Y%m%d')}.csv", append=True))

    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = m.fit(X_train, Y_train, callbacks=callbacks_used, verbose=2, **kwargs)
    if save_final:
        make_dir_if_not_exist(model_name)
        m.save(f"/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}_saved_model_after_fit")  # Save the model
    return (m, history)


In [11]:
def compile_and_fit_model_collaborative(  
        m1, m1_layer_name,
        m2, m2_layer_name,
        input_shape,
        X_train,
        Y_train,
        X_validation,
        Y_validation,
        X_test,
        Y_test,
        save_max_epoch=True,
        save_final=False,
        patience_count = None,
        log_history = True,
        **kwargs):
    
    to_combine_ld_no2nd_rNo2nd = [
        (m1 , m1_layer_name, None),   # change to the appropriate dense layer name
        (m2, m2_layer_name, None) # change to the appropriate dense layer name
    ]
    
    # models, data_train, data_validation, data_test, XY_dict
    combined_models_ld, data_train_ld, data_validation_ld, data_test_ld, data_access_ld = get_combined_features_from_models(
        to_combine_ld_no2nd_rNo2nd ,
        X_train,
        Y_train,
        X_validation,
        Y_validation,
        X_test, 
        Y_test, 
        reverse_one_hot=False)
        
    rcnn_combine_model = model_combination(f"collaborative_model_{m1.name}_{m2.name}_{datetime.datetime.now():%Y%m%d%H%M%S}", 
                                           data_train_ld[0][0].shape  )
    rcnn_combine_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
    if patience_count is None:
        patience_count  = 10000    
    callbacks_used_rcnn_combine = [
                        ModelCheckpoint(f'/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{rcnn_combine_model.name}' + '_model_{epoch:03d}_{accuracy:0.3f}',
                                                save_weights_only=False,
                                                monitor='accuracy',
                                                mode='max',
                                                save_best_only=True),
                        tf.keras.callbacks.EarlyStopping(patience=patience_count),
                        tf.keras.callbacks.CSVLogger(f"/content/drive/MyDrive/ncRNA/data/history/history_log_{rcnn_combine_model.name}_{datetime.date.today().strftime('%Y%m%d')}.csv", append=True)
                        ]
    
    history_rcnn_combine = rcnn_combine_model.fit(data_train_ld[0], 
                                                  data_train_ld[1], 
                                                  validation_data=(data_validation_ld[0],to_categorical(data_validation_ld[1]-1)),
                                                  callbacks=callbacks_used_rcnn_combine, 
                                                  verbose=2, 
                                                  **kwargs)
    rcnn_combine_model.save(f"/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{rcnn_combine_model.name}.h5")    
    # rcnn_combine_model.evaluate(data_test_ld_no2nd_rNo2nd[0],data_test_ld_no2nd_rNo2nd[1][0]) 
    
    return (rcnn_combine_model, history_rcnn_combine, data_access_ld )

In [12]:
# 'new' data 
X_train_1000e, Y_train_1000e, X_test_1000e, Y_test_1000e, X_val_1000e, Y_val_1000e = getE2eData(is500=False,
                                                                                                    include_secondary=False,
                                                                                                    isColab=True)
X_train_1000e_w2nd, Y_train_1000e_w2nd, X_test_1000e_w2nd, Y_test_1000e_w2nd, X_val_1000e_w2nd, Y_val_1000e_w2nd = getE2eData(is500=False, include_secondary=True, isColab=True)
X_train_1000e_j2nd, Y_train_1000e_j2nd, X_test_1000e_j2nd, Y_test_1000e_j2nd, X_val_1000e_j2nd, Y_val_1000e_j2nd = getE2eDataJustSecondary(isColab=True)

In [13]:
# merge into a new train:
X_new_train = np.concatenate( (X_train_1000e, X_val_1000e), axis=0 )
Y_new_train = np.concatenate( (Y_train_1000e, Y_val_1000e), axis=0 )    

X_new_train_j2nd = np.concatenate( (X_train_1000e_j2nd, X_val_1000e_j2nd), axis=0 )
Y_new_train_j2nd = np.concatenate( (Y_train_1000e_j2nd, Y_val_1000e_j2nd), axis=0 )    

X_new_train_w2nd = np.concatenate( (X_train_1000e_w2nd, X_val_1000e_w2nd), axis=0 )
Y_new_train_w2nd = np.concatenate( (Y_train_1000e_w2nd, Y_val_1000e_w2nd), axis=0 )  

In [14]:
print(X_new_train.shape)

(7715, 1000, 8)


In [ ]:
# # Continue model E run from before...
rnn_E = load_model(f"/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/rnn_E_20210827221034_model_119_0.979")
m = rnn_E
patience_count=60
callbacks_used = []
callbacks_used.append(ModelCheckpoint(f'/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}' + '_model_{epoch:03d}_{accuracy:0.3f}',
                                      save_weights_only=False,
                                      monitor = 'accuracy', # 'val_accuracy'
                                      mode='max',
                                      save_best_only=True))
callbacks_used.append(tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=patience_count))
callbacks_used.append(tf.keras.callbacks.CSVLogger(f"/content/drive/MyDrive/ncRNA/data/history/history_log_{m.name}_{datetime.date.today().strftime('%Y%m%d')}.csv", append=True))
# check if continues from before..

m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn_E = m.fit(X_train_1000e, Y_train_1000e, callbacks=callbacks_used, verbose=2,                                                   
                      batch_size=128,
                      epochs=500-118-6-119,
                      class_weight=None,
                      validation_data=(X_val_1000e, Y_val_1000e))
m.save(f"/content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/{m.name}_saved_model_after_fit")  # Save the model

Epoch 1/257
54/54 - 786s - loss: 0.0862 - accuracy: 0.9736 - val_loss: 0.4591 - val_accuracy: 0.9032
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/rnn_E_20210827221034_model_001_0.974/assets
Epoch 2/257
54/54 - 768s - loss: 0.0854 - accuracy: 0.9752 - val_loss: 0.4057 - val_accuracy: 0.9148
INFO:tensorflow:Assets written to: /content/drive/MyDrive/data_papers/ncRNA/model_checkpoints/rnn_E_20210827221034_model_002_0.975/assets
Epoch 3/257
54/54 - 742s - loss: 0.0934 - accuracy: 0.9720 - val_loss: 0.5057 - val_accuracy: 0.9043
Epoch 4/257
54/54 - 758s - loss: 0.0867 - accuracy: 0.9711 - val_loss: 0.3987 - val_accuracy: 0.9265
Epoch 5/257
54/54 - 720s - loss: 0.0892 - accuracy: 0.9726 - val_loss: 0.4499 - val_accuracy: 0.9137
Epoch 6/257
54/54 - 751s - loss: 0.0795 - accuracy: 0.9742 - val_loss: 0.2980 - val_accuracy: 0.9347
Epoch 7/257
54/54 - 773s - loss: 0.0895 - accuracy: 0.9717 - val_loss: 0.4820 - val_accuracy: 0.9067
Epoch 8/257
54/54